In [4]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

# tensorboard
# 1.指定面板图上显示的变量
# 2.训练过程中将这些变量计算出来,输出到文件中
# 3.文件解析 ./tensorboard --logdir=dir.

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [5]:
def load_data(filename):
    """read data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='iso-8859-1')
        return data['data'],data['labels']

#tensorflow.Dataset
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def next_batch(self,batch_size):
        """:return batch_szie examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i ) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,True)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [6]:
batch_size = 20
X = tf.placeholder(tf.float32, [batch_size, 3072])
y = tf.placeholder(tf.int64, [batch_size])
is_training = tf.placeholder(tf.bool, [])
# [None], eg: [0,5,6,3]
X_image = tf.reshape(X, [-1,3,32,32])
# 32 * 32
X_image = tf.transpose(X_image, perm = [0,2,3,1])

X_image_arr = tf.split(X_image, num_or_size_splits = batch_size, axis = 0)
result_X_image_arr = []

for X_single_image in X_image_arr:
    # X_single_image:[1, 32, 32, 3] -> [32,32,3]
    X_single_image = tf.reshape(X_single_image, [32, 32, 3])
    data_aug_1 = tf.image.random_flip_left_right(X_single_image)
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta = 63)
    data_aug_3 = tf.image.random_contrast(data_aug_2, lower = 0.2, upper = 1.8)
    X_single_image = tf.reshape(data_aug_3, [1, 32, 32, 3])
    result_X_image_arr.append(X_single_image)
# 合并为mini batch
result_X_images = tf.concat(result_X_image_arr, axis = 0)

# 归一化
normal_result_X_images = result_X_images / 127.5 - 1

# conv1: 神经元图, feature_map, 输出图像
with tf.variable_scope("encoder",reuse=tf.AUTO_REUSE) as scope:
    def conv_wrapper(inputs,
                     name,
                     is_training,# 判断是否在训练
                     output_channel = 32,
                     kernel_size = (3,3),
                     padding = 'same',
                     activation = tf.nn.relu):
        """
        wrapper of tf.layers.conv2d
        :param inputs:
        :param name:
        :param output_channel:
        :param kernel_size:
        :param padding:
        :param activation:
        :return:
        """
        # without bn: conv -> activation
        # with batch normalization: conv -> bn -> activation
        with tf.name_scope(name):
            conv2d = tf.layers.conv2d(inputs,
                                      output_channel,# output channel number
                                      kernel_size,# kernel size
                                      padding = padding,
                                      activation = None,
                                      reuse = tf.AUTO_REUSE,
                                      name = name + '/conv2d')
            bn = tf.layers.batch_normalization(conv2d,
                                               training = is_training)
            return activation(bn)


    def pooling_wrapper(inputs, name):
        """
        wrapper of tf.layers.max_pooling2d
        :param inputs:
        :param name:
        :return:
        """
        return tf.layers.max_pooling2d(inputs,
                                   (2,2), # kernel size
                                   (2,2), #stride
                                   name = name)

    conv1_1 = conv_wrapper(normal_result_X_images, 'conv1_1', is_training)
    conv1_2 = conv_wrapper(conv1_1, 'conv1_2', is_training)
    conv1_3 = conv_wrapper(conv1_2, 'conv1_3', is_training)
    pooling1 = pooling_wrapper(conv1_3, 'pool1')

    conv2_1 = conv_wrapper(pooling1, 'conv2_1', is_training)
    conv2_2 = conv_wrapper(conv2_1, 'conv2_2', is_training)
    conv2_3 = conv_wrapper(conv2_2, 'conv2_3', is_training)
    pooling2 = pooling_wrapper(conv2_3, 'pool2')

    conv3_1 = conv_wrapper(pooling2, 'conv3_1', is_training)
    conv3_2 = conv_wrapper(conv3_1, 'conv3_2', is_training)
    conv3_3 = conv_wrapper(conv3_2, 'conv3_3', is_training)
    pooling3 = pooling_wrapper(conv3_3, 'pool3')

    # [None, 4 * 4 * 32]
    flatten = tf.layers.flatten(pooling3)

    y_ = tf.layers.dense(flatten, 10)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
    # y_ -> softmax
    # y -> one_hot
    # loss = ylogy_

    # indices
    predict = tf.argmax(y_,1)
    # [1,0,1,1,1,0,0,0]
    correct_prediction = tf.equal(predict,y)
    accuary = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

    with tf.name_scope('train_op'):
        # 反向传播
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [7]:
def variable_summary(var, name):
    """
    给一个变量的很多统计量建立summary
    :param var:
    :param name:
    :return:
    """
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean',mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.histogram('histogram',var)

with tf.name_scope('summary'):
    variable_summary(conv1_1,'conv1_1')
    variable_summary(conv1_2,'conv1_2')
    variable_summary(conv2_1,'conv2_1')
    variable_summary(conv2_2,'conv2_2')
    variable_summary(conv3_1,'conv3_1')
    variable_summary(conv3_2,'conv3_2')
loss_summary = tf.summary.scalar('loss',loss)
# 'loss' : <10,1.1>,<20,1.08>
accuracy_summary = tf.summary.scalar('accuracy',accuary)

inputs_summary = tf.summary.histogram('inputs_image',normal_result_X_images)

# 合并summary
merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')

if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

init = tf.global_variables_initializer()
train_steps = 1000
test_steps = 100
output_summary_every_steps = 100

with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir,sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)

    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuary, train_op]
        should_output_summary = ((i + 1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)

        eval_ops_result = sess.run(
            eval_ops,
            feed_dict = {
                X: batch_data,
                y: batch_labels,
                is_training: True
            })
        loss_val, acc_val = eval_ops_result[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_result[-1]
            train_writer.add_summary(train_summary_str,i + 1)
            test_summary_str = sess.run([merged_summary_test],
                                        feed_dict = {
                                            X: fixed_test_batch_data,
                                            y: fixed_test_batch_labels,
                                            is_training: False
                                        })[0]
            test_writer.add_summary(test_summary_str, i + 1)

        if (i+1) % 100 == 0:
            print('[Train] Step :%d, loss: %4.5f, acc: %4.5f'\
                  %(i+1,loss_val,acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data ,test_batch_labels\
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuary],
                    feed_dict = {
                        X: test_batch_data,
                        y: test_batch_labels,
                        is_training: False
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.nanmean(all_test_acc_val)
            print('[Test ] Step :%d, acc: %4.5f'\
                  %(i+1,test_acc))


[Train] Step :100, loss: 1.82121, acc: 0.45000
[Train] Step :200, loss: 1.76186, acc: 0.15000
[Train] Step :300, loss: 1.92618, acc: 0.25000
[Train] Step :400, loss: 1.32974, acc: 0.45000
[Train] Step :500, loss: 1.71770, acc: 0.35000
[Train] Step :600, loss: 1.68627, acc: 0.45000
[Train] Step :700, loss: 1.56937, acc: 0.45000
[Train] Step :800, loss: 1.64385, acc: 0.40000
[Train] Step :900, loss: 1.43116, acc: 0.45000
[Train] Step :1000, loss: 1.78048, acc: 0.35000
(10000, 3072)
(10000,)
[Test ] Step :1000, acc: 0.19200
